<a href="https://colab.research.google.com/github/wittyicon29/GenreGenie-A-Movie-genre-prediction-model/blob/main/models/BERT_architecture.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install --upgrade pandas pyarrow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.3/12.3 MB 54.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.9/38.9 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 341.8/341.8 kB 43.2 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 9.0.0
    Uninstalling pyarrow-9.0.0:
      Successfully uninstalled pyarrow-9.0.0
  Attempting uninstall: pandas
    Found existing installation: pandas 1.5.3
    Uninstalling pandas-1.5.3:
      Successfully uninstalled pandas-1.5.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires pandas==1.5.3, but you have pandas 2.0.3 which is incompatible.
pandas-gbq 0.17.9 requires pyarrow<10.0dev,>=3.0.0, but you have pyarrow 12.0.1 which is incompatible.


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet("/content/train-00000-of-00001-b943ea66e0040b18.parquet")

In [ ]:
data_df = pd.DataFrame(df)
data_df

,id,movie_name,synopsis,genre
0,44978,Super Me,A young scriptwriter starts bringing valuable ...,fantasy
1,50185,Entity Project,A director and her friends renting a haunted h...,horror
2,34131,Behavioral Family Therapy for Serious Psychiat...,This is an educational video for families and ...,family
3,78522,Blood Glacier,Scientists working in the Austrian Alps discov...,scifi
4,2206,Apat na anino,Buy Day - Four Men Widely - Apart in Life - By...,action
...,...,...,...,...
53995,80325,Untitled Monkeyshine/Ugly Brother Project,Two loser pest control buddies find themselves...,scifi
53996,84712,The Great Swindle,A seductive woman gets involved in relationshi...,thriller
53997,69801,When Men Get Pregnant,"Duyen, a wedding dress staff, who decides to c...",romance
53998,7678,Suttu Pidikka Utharavu,The people of a crowded colony in Coimbatore c...,action


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

In [ ]:
X = data_df['synopsis'].values
y = data_df['genre'].values

In [ ]:
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

In [ ]:
X_train, X_val, y_train, y_val = train_test_split(X, y_encoded, test_size=0.2, random_state=42)

In [ ]:
!pip install transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 100.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 64.0 MB/s eta 0:00:00


In [ ]:
import torch
from torch.utils.data import DataLoader, Dataset
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score, classification_report
import numpy as np

In [ ]:
# Assuming y_train is your target labels for the training set
num_classes = len(set(y_train))
print("Number of classes:", num_classes)

Number of classes: 10


In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=num_classes) # num_classes is the number of movie genres

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
class MovieGenreDataset(Dataset):
    def __init__(self, X, y, tokenizer, max_len=128):
        self.X = X
        self.y = y
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        text = self.X[idx]
        label = self.y[idx]

        encoding = self.tokenizer(text, padding='max_length', truncation=True, max_length=self.max_len, return_tensors='pt')

        input_ids = encoding['input_ids'].flatten()
        attention_mask = encoding['attention_mask'].flatten()
        token_type_ids = encoding['token_type_ids'].flatten()
        label = torch.tensor(label, dtype=torch.long)

        return {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
            'token_type_ids': token_type_ids,
            'label': label
        }

In [ ]:
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [ ]:
train_dataset = MovieGenreDataset(X_train, y_train_encoded, tokenizer)
val_dataset = MovieGenreDataset(X_val, y_val_encoded, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32)

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [ ]:
optimizer = AdamW(model.parameters(), lr=2e-5)
loss_fn = torch.nn.CrossEntropyLoss()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


In [ ]:
num_epochs = 3
for epoch in range(num_epochs):
    model.train()
    train_loss = 0.0
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label = batch['label'].to(device)

        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids, labels=label)
        loss = outputs.loss
        loss.backward()
        optimizer.step()

        train_loss += loss.item()

    train_loss /= len(train_loader)
    print(f'Epoch {epoch+1}/{num_epochs}, Training Loss: {train_loss:.4f}')

Epoch 1/3, Training Loss: 1.7488
Epoch 2/3, Training Loss: 1.5105
Epoch 3/3, Training Loss: 1.2839


In [ ]:
model.eval()
val_predictions = []
val_targets = []
with torch.no_grad():
    for batch in val_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)
        label = batch['label'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        val_predictions.extend(predictions.cpu().numpy())
        val_targets.extend(label.cpu().numpy())

val_predictions = label_encoder.inverse_transform(val_predictions)
val_targets = label_encoder.inverse_transform(val_targets)

In [ ]:
accuracy = accuracy_score(val_targets, val_predictions)
classification_report_str = classification_report(val_targets, val_predictions)

print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:")
print(classification_report_str)

Validation Accuracy: 0.3756
Classification Report:
              precision    recall  f1-score   support

           0       0.31      0.30      0.31      1094
           1       0.30      0.26      0.28      1067
           2       0.37      0.39      0.38      1134
           3       0.47      0.44      0.45      1049
           4       0.33      0.32      0.32      1057
           5       0.42      0.45      0.44      1116
           6       0.30      0.32      0.31      1074
           7       0.56      0.55      0.56      1075
           8       0.42      0.52      0.47      1077
           9       0.23      0.20      0.21      1057

    accuracy                           0.38     10800
   macro avg       0.37      0.38      0.37     10800
weighted avg       0.37      0.38      0.37     10800



In [ ]:
test_df = pd.read_parquet("/content/test-00000-of-00001-35e9a9274361daed.parquet")

In [ ]:
test_df = pd.DataFrame(test_df)
test_df

,id,movie_name,synopsis,genre
0,16863,A Death Sentence,"12 y.o. Ida's dad'll die without a DKK1,500,00...",action
1,48456,Intermedio,A group of four teenage friends become trapped...,action
2,41383,30 Chua Phai Tet,A guy left his home for 12 years till he came ...,action
3,84007,Paranoiac,A man long believed dead returns to the family...,action
4,40269,Ordinary Happiness,"After a deadly accident, Paolo comes back on E...",action
...,...,...,...,...
35995,73156,Mysterious Doctor Satan,A mad scientist plots the domination of Americ...,action
35996,21242,Blondes at Work,Steve promises his captain not to favor Torchy...,action
35997,70135,The Hurricane,A Polynesian sailor unjustly imprisoned after ...,action
35998,85987,Malayankunju,Anikkuttan is an ill-tempered electronics tech...,action


In [ ]:
test_dataset = MovieGenreDataset(test_df['synopsis'].values, np.zeros(len(test_df)), tokenizer)
test_loader = DataLoader(test_dataset, batch_size=32)

In [ ]:
test_predictions = []
with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        token_type_ids = batch['token_type_ids'].to(device)

        outputs = model(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=1)

        test_predictions.extend(predictions.cpu().numpy())

In [ ]:
test_predictions = label_encoder.inverse_transform(test_predictions)

In [ ]:
sample_submission_file = "/content/sample_submission.csv"
sample_submission_df = pd.read_csv(sample_submission_file)

In [ ]:
test_predictions_bert = pd.DataFrame({'id': range(1, len(test_predictions)+1), 'genre': test_predictions})

In [ ]:
merged_df_bert = pd.merge(sample_submission_df, test_predictions_bert, on='id', suffixes=('_sample', '_your'))

In [ ]:
merged_df_bert['genre_sample'] = merged_df_bert['genre_your']

In [ ]:
final_submission_file = "/content/final_submission.csv"
merged_df_bert.to_csv(final_submission_file, columns=['id', 'genre_sample'], index=False)

In [ ]:
num_to_genre = {
    0: 'action',
    1: 'adventure',
    2: 'crime',
    3: 'family',
    4: 'fantasy',
    5: 'horror',
    6: 'mystery',
    7: 'romance',
    8: 'scifi',
    9: 'thriller'
}

In [ ]:
predictions_file = "/content/final_submission.csv"
df = pd.read_csv(predictions_file)
df.columns

Index(['id', 'genre_sample'], dtype='object')

In [ ]:
df['genre_sample'] = df['genre_sample'].map(num_to_genre)

In [ ]:
output_file = "/content/final.csv"
df.to_csv(output_file, index=False)